## Ingest images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from buck.analysis.basics import ingest_images

fpath = "..\\images\\squared\\*.png"
images,ages = ingest_images(fpath)

## Split datasets

In [ ]:
from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

## Homogenize data across classes

In [ ]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 25
X_train_pca, y_train_flat, X_test_pca, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

## Optimize all classifier models

In [ ]:
from buck.classifiers.autotune import optimize_all

results = optimize_all(X_train_pca, y_train_flat, X_test_pca, y_true, cycles = 5)
print(results)